In [3]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import h5py

from matplotlib import animation
from matplotlib import cm
from mpl_toolkits import mplot3d
import os,sys

sys.path.append('/Users/ajc/Core/PyScript')
import readdata as rd
import os
import seaborn as sns
import geopy.distance
from matplotlib import cm

In [4]:
mesh_resfile= '/Users/ajc/Core/simulations/production/streams/SIM_13/mesh_coords_10m/ats_vis_surface_mesh.h5'
mesh_hres = rd.get_mesh_coords(mesh_resfile)

In [5]:
sim = 'S6_dx1000m'
sim_mesh= ''
path='/Users/ajc/Core/simulations/production/streams/SIM_13/%s'%sim

outfile='/Users/ajc/Core/PostProcessing/2020/StreamHyporheic/Dec8/SIM_09/%s/'%sim
infile = [os.path.join(path,'ats_vis_surface_data.h5'), os.path.join(path,'ats_vis_surface_mesh.h5') ]

In [ ]:
mesh = rd.get_mesh_coords(infile[1])

In [ ]:
#variables = [(0,'DOM1'), (1,'O2(aq)'), (2,'NO3-'), (3,'HCO3-'),(4,'H+'), (5,'N2(aq)')]
variables = [(0,'Tracer1'), (1,'Tracer2')]
#var = 'DOM1 conc'
var_c = 0
var = variables[var_c][0]
C_data = rd.GetSurfVarFromVis_All(infile[0],'surface-total_component_concentration',suffix='.cell.%s'%var)
Concentration = C_data['surface-total_component_concentration']
cycles = len(Concentration)
print (cycles)


In [ ]:
def get_limits():
    fig, ax = plt.subplots(1,2,figsize=(10,3))#,subplot_kw={"projection": "3d"})
    #fig = plt.figure(figsize=(20,8))
    #ax = fig.add_subplot(121)

    ax[0].scatter(mesh['x'],mesh['y'],marker=".",c=mesh['z'])
    #ax.set_xlim(xmin-0.0005,xmax)
    xmin, xmax = ax[0].get_xlim()
    ymin, ymax = ax[0].get_ylim()
    print(xmin,xmax)
    print(ymin,ymax)
    # Coweeta
    #X0 = [-83.48, -83.40]
    #Y0 = [35.03, 35.08]
    X0 = [-84.41, -84.22]
    Y0 = [35.92, 36.03]
    #PR
    #X0 = [-84.22, -84.4]
    #Y0 = [35.92, 36.025]
    ax[0].set_xlim(X0)
    ax[0].set_ylim(Y0)
    xmin, xmax = ax[0].get_xlim()
    ymin, ymax = ax[0].get_ylim()

    xx = geopy.distance.geodesic(xmin,xmax).km + 0.07
    yy = geopy.distance.geodesic(ymin,ymax).km + 0.455
    Xlabels = np.round(np.linspace(0,xx,10,dtype=int),2)
    Ylabels = np.round(np.linspace(0,yy,7,dtype=int),2)

    #ax = fig.add_subplot(122)
    ax[1].scatter(mesh['x'],mesh['y'],marker=".",c=mesh['z'])
    #print (len(X_mesh),len(Y_mesh),len(Z_mesh))
    #ax[1].plot_surface(X_mesh, Y_mesh, [Z_mesh,Z_mesh], cmap=cm.coolwarm,linewidth=0, antialiased=False)
    ax[1].set_xticklabels(Xlabels)
    ax[1].set_yticklabels(Ylabels)
    ax[1].set_xlim(xmin,xmax)
    ax[1].set_ylim(ymin,ymax)
    #plt.gca().invert_xaxis()
    plt.show()
    return X0, Y0, Xlabels, Ylabels
    
X0, Y0, Xlabels,Ylabels = get_limits()


In [ ]:
#Write solute concentration images to a directory
%matplotlib qt

#x_orig = X_mesh
#y_orig = Y_mesh

centroid = []
thick = [1,1,1]
count = 0

time = [np.round(t*360/86400,2) for t in range(cycles)]
RR = []
to_micro = 1#1e+6
start = 1400
end = start + 1
for k in range(start,cycles,1):
    var = variables[var_c][1]
    print (var, var_c)
    fig, ax = plt.subplots(1,1,figsize=(6,4))
    
    ran = Concentration[k]
    
    ran = np.array([r*1e-3*to_micro if r>0 else 1e-12 for r in ran])
    
    #print (ran)
    xx = np.where(ran == np.min(ran))
    #print (ran)
    print ('Max: ',len(ran),np.max(ran), np.min(ran), xx)#, ran)#, np.min(ran), xx)#, ran[pos],ran.shape)
    
    l_max = 1#100
    """
    if var == 'NO3-':
        if np.max(ran) <51 or ran[0]=='inf':
            l_max = 50
        else:
            l_max = 500
    elif var == 'DOM1':
        if np.max(ran) <401 or ran[0]=='inf':
            #print (np.max(ran))
            #print (ran[180:200])
            l_max = np.max(ran)#100
        else:
            l_max = np.max(ran)#1000
    """
    #ax.scatter(mesh_hres['x'],mesh_hres['y'],marker=".",c='lightgrey',s=1) # mesh resolution
    ss = 0 #892
    ll = -1#217
    sc = ax.scatter(mesh['x'][ss:ll],mesh['y'][ss:ll],marker=".",c=ran[ss:ll],cmap=plt.get_cmap('viridis'),vmin=1.e-4,vmax=l_max,norm=matplotlib.colors.LogNorm())
    ax.text(mesh['x'][-1]-0.02,mesh['y'][-1]+0.075, 'Time = %s [hr]'%time[k],fontsize=12,fontstyle='normal')
    cb = plt.colorbar(sc,pad=0.02,fraction=0.046)
    cb.ax.set_ylabel(r'Concentration [M/liter]',labelpad=-70, y=1.1,rotation=0,fontsize=12)
    #cmap=plt.get_cmap('jet')
    RR.append(ran)

    
    ax.set_xlim(X0)
    ax.set_ylim(Y0)
    xmin, xmax = ax.get_xlim()
    ymin, ymax = ax.get_ylim()
    ax.set_xticklabels(Xlabels)
    ax.set_yticklabels(Ylabels)
    ax.set_xlabel('Distance [km]',fontsize=12, fontweight='normal')
    ax.set_ylabel('Distance [km]',fontsize=12, fontweight='normal')
    plt.tight_layout(h_pad=1)
    #plt.gca().invert_xaxis()
    #plt.text(X_mesh[-1]-0.02,Y_mesh[-1]+0.025, 'Time [hr] = %s'%time[k])
    
    
    #plt.savefig(outfile + '/%s/fig-%s.png'%(var,k),dpi=100)
    
    count =1 + k
    if count >end:
        break
    plt.close()
#print (l_max)

In [ ]:
import re, os

def sorted_nicely( l ):
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key = alphanum_key)

path_anim = os.path.join(outfile,var) + '/'
dir_files = os.listdir(path_anim)

Files = sorted_nicely(dir_files)
print (path_anim,outfile)
Files = [f for f in Files if f.endswith('.png')]
print (Files[:2])
import cv2
import numpy as np
#Files = Files[:60] + Files[450:520] #+ Files[37:70]
img_array = []
for filename in Files[::1]:
    #print (filename)
    filename = path_anim + filename
    #print (filename)
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
    


In [ ]:
out = cv2.VideoWriter(path_anim + 'RT-coweeta.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 5, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

In [ ]:
implied_s = h5py.File('/Users/ajc/FUSE/simulations/tests/ats-demos/13_reactive_transport/04_hyporheic_watershed/coweeta_data/implied_source/coweeta-cellsource-100m.h5','r')
implied_s = h5py.File('/Users/ajc/FUSE/simulations/tests/ats-demos/13_reactive_transport/04_hyporheic_watershed/coweeta_data/implied_source/coweeta-impliedsource_watershed-100m.h5','r')
#print (implied_s.keys())
implied_src = np.concatenate(implied_s['surface-mass_src.cell.0']['0'])
#print (implied_src)

discharge_s = h5py.File('/Users/ajc/FUSE/simulations/tests/ats-demos/13_reactive_transport/04_hyporheic_watershed/coweeta_data/discharge/coweeta-faceQ-100m.h5','r')
discharge = np.concatenate(discharge_s['surface-mass_flux.face.0']['0'])
discharge = [d for d in discharge if d >0]

cross_area = np.array([[0.8163220080526847,]*30, [0.36615236094580733,]*7, [0.4004007127485738,]*11, [0.3363141193639137,]*3,
             [0.03006694550743245,]*10,[0.1924396943885528,]*20, [0.12654463237568553,]*30, [0.03935396817517269,]*9,
             [0.07236637782749487,]*17,[0.06851824173774428,]*12, [0.035624209712113754,]*11, [0.33945737398362685,]*5,
             [0.24954923641757704,]*2,[0.09095085110858311,]*8, [0.04622924887712758,]*13, [0.20248899279870614,]*10,
             [0.1641051190892562,]*8,[0.020599246486715468,]*8, [0.07199534283806251,]*11, [0.0651169621600101,]*12,
             [0.024979953950850804,]*9,[0.06276187165492346,]*14])

cross_area = np.concatenate(cross_area)

Length = np.array([[25, 72.4138], [97.4138,]*28, [88.4286,]*7, [99.2727,]*11, [92.3333,]*3,
             [94.5,]*10,[98.5,]*20, [96.6667,]*30, [91.0,]*9,
             [97.3529,]*17,[97.3333,]*12, [95.0,]*11, [84.2,]*5,
             [82.5,]*2,[96.875,]*8, [99.3077,]*13, [98.7,]*10,
             [95.625,]*8,[98.375,]*8, [94.0909,]*11, [97.75,]*12,
             [96.7778,]*9,[97.2857,]*14])

Length = np.concatenate(Length)
Volume = np.array([r1*r2 for r1,r2 in zip(Length, cross_area)])
print (len(cross_area))

In [ ]:
1/(9.8*1000)*1

In [ ]:
2 meadian life time , 2.6hr give mean of 1hr median
alpha = 2.5 1/sec